## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,57.79,79,20,6.62,few clouds
1,1,Hervey Bay,AU,-25.2986,152.8535,71.02,77,66,5.01,broken clouds
2,2,New Norfolk,AU,-42.7826,147.0587,54.72,62,100,1.01,overcast clouds
3,3,Narsaq,GL,60.9167,-46.0500,41.61,57,74,6.22,broken clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,64.40,55,75,10.36,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Lorengau,PG,-2.0226,147.2712,86.02,66,100,11.99,overcast clouds
15,15,Mahibadhoo,MV,3.7833,72.9667,80.02,80,100,16.67,overcast clouds
21,21,Hasaki,JP,35.7333,140.8333,88.29,79,5,23.15,clear sky
28,28,Cabo San Lucas,MX,22.8909,-109.9124,86.50,73,100,1.92,overcast clouds
29,29,Hami,CN,42.8000,93.4500,77.88,22,100,9.82,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Lorengau,PG,86.02,overcast clouds,-2.0226,147.2712,
15,Mahibadhoo,MV,80.02,overcast clouds,3.7833,72.9667,
21,Hasaki,JP,88.29,clear sky,35.7333,140.8333,
28,Cabo San Lucas,MX,86.50,overcast clouds,22.8909,-109.9124,
29,Hami,CN,77.88,overcast clouds,42.8000,93.4500,
31,Victoria,HK,89.33,broken clouds,22.2855,114.1577,
35,Butaritari,KI,82.26,clear sky,3.0707,172.7902,
36,Aitape,PG,82.62,broken clouds,-3.1374,142.3475,
37,Matara,LK,76.44,overcast clouds,5.9485,80.5353,
42,Jimo,CN,77.41,clear sky,36.3897,120.4622,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        
    except (IndexError):
        
        print("Hotel not found... skipping")   

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [31]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.count()

City                   231
Country                231
Max Temp               231
Current Description    231
Lat                    231
Lng                    231
Hotel Name             231
dtype: int64

In [30]:
# 8a. Create the output File (CSV)

output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Description</dt><dd>{Current Description}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))